# Tensorflow tutorial with titanic dataset
## Basic model training

In [7]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [8]:
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

In [9]:
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Perform a calculation using the input
result = 2*input + 1

# the result doesn't have a value
result

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.add')>

In [10]:
calc = tf.keras.Model(inputs=input, outputs=result)
print(calc(1).numpy())
print(calc(2).numpy())

3.0
5.0


To build the preprocessing model, start by building a set of symbolic keras.Input objects, matching the names and data-types of the CSV columns.

In [11]:
inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'n_siblings_spouses': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'n_siblings_spouses')>,
 'parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'parch')>,
 'fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fare')>,
 'class': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'class')>,
 'deck': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'deck')>,
 'embark_town': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'embark_town')>,
 'alone': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'alone')>}

The first step in your preprocessing logic is to concatenate the numeric inputs together, and run them through a normalization layer:

In [12]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization')>

In [13]:
# Collect all the symbolic preprocessing results, to concatenate them later.

preprocessed_inputs = [all_numeric_inputs]

For the string inputs use the tf.keras.layers.StringLookup function to map from strings to integer indices in a vocabulary. Next, use tf.keras.layers.CategoryEncoding to convert the indexes into float32 data appropriate for the model. 

In [14]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [15]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

titanic_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

titanic_features_dict = {name: np.array(value) 
                         for name, value in titanic_features.items()}

features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
titanic_preprocessing(features_dict)

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[-0.61 ,  0.395, -0.479, -0.497,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  1.   ,  0.   ]], dtype=float32)>

In [16]:
# build the model

def titanic_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam())
  return model

titanic_model = titanic_model(titanic_preprocessing, inputs)

## Train model

In [17]:
devices = tf.config.list_physical_devices()

In [25]:
with tf.device('/GPU:0'):
    titanic_model.fit(x=titanic_features_dict, y=titanic_labels, epochs=200)

Epoch 1/200
20/20 [==============================] - 0s 13ms/step - loss: 0.4172
Epoch 2/200
20/20 [==============================] - 0s 12ms/step - loss: 0.4171
Epoch 3/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4175
Epoch 4/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4180
Epoch 5/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4174
Epoch 6/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4178
Epoch 7/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4184
Epoch 8/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4178
Epoch 9/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4176
Epoch 10/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4190
Epoch 11/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4176
Epoch 12/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4180
Epoch 13/200
20/20 [==========

In [26]:
with tf.device('/CPU:0'):
    titanic_model.fit(x=titanic_features_dict, y=titanic_labels, epochs=200)

Epoch 1/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4174
Epoch 2/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4169
Epoch 3/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4179
Epoch 4/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4169
Epoch 5/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4176
Epoch 6/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4187
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4180
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4177
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4177
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4174
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4174
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4182
Epoch 13/200
20/20 [=================

## Optionally print model architecture diagram

In [ ]:
# preprocessing
tf.keras.utils.plot_model(model = titanic_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:
# actual model
tf.keras.utils.plot_model(model = titanic_model , rankdir="LR", dpi=72, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
